![anuncio](../static/imgs/MagicMovieMachine_banner.png)
# Creación de su propio recomendador de películas

Ahora es el momento de crear su propio recomendador de películas en su cuenta de AWS. En este cuaderno de Jupyter,
verá cómo puede crear recomendadores que generen recomendaciones, como Magic Movie Machine.

En primer lugar, descargará, modificará e importará los datos de las películas recopilados del conjunto de datos del [proyecto de MovieLens](https://grouplens.org/datasets/movielens/) en Amazon Personalize. A continuación, Amazon Personalize utilizará estos datos para crear recomendadores: uno para el caso de uso *Principales selecciones para usted* y otro para el caso de uso *Más como X*. 

Estos son casos de uso preconfigurados que generan recomendaciones. En segundo plano, Amazon Personalize entrena múltiples modelos específicos de casos de uso para cada recomendador. *Principales selecciones para usted* genera recomendaciones similares a las mejores recomendaciones que vio en Magic Movie Machine. *Más como X* es una versión más sencilla de las recomendaciones de *Joyas ocultas*: Genera recomendaciones de películas similares, pero que no son joyas menos conocidas.

Por último, obtendrá recomendaciones de películas, como las que vio en Magic Movie Machine.

## Costo y duración del tutorial
Este tutorial puede tardar alrededor de 2 horas en completarse. Según su cuenta de AWS, podría incurrir en algún costo. Si utiliza el nivel gratuito de AWS, complete el tutorial el mismo día que comienza y elimine cualquier recurso cuando finalice. Los costos serán mínimos o nulos. Para obtener una lista completa de los cargos y los precios, consulte [precios de Amazon Personalize](https://aws.amazon.com/personalize/pricing/).

Para evitar incurrir en gastos innecesarios, asegúrese de ejecutar el cuaderno `Clean_Up_Resources.ipynb` que se encuentra en este repositorio.
# Cómo utilizar el cuaderno

El código se divide en celdas como la siguiente. En la parte superior de esta página, puede hacer clic en un botón triangular para Ejecutar cada celda y pasar a la siguiente, o puede presionar `Shift` + `Enter` mientras está en la celda para ejecutarla y pasar a la siguiente.

A medida que una celda se ejecuta, verá una línea al costado que muestra un `*` mientras la celda se está ejecutando o se actualizará a un número para indicar la última celda que terminó de ejecutarse luego de que haya terminado de ejecutar todo el código dentro de una celda.

Solo debe seguir las instrucciones que aparecen a continuación y ejecutar las celdas para comenzar a utilizar Amazon Personalize con recomendadores optimizados para el caso.

## Importaciones
Python incluye una amplia colección de bibliotecas. Es necesario importar estas bibliotecas y otros paquetes adicionales como [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK para Python) y [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), que son herramientas centrales de la ciencia de datos.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

A continuación, deberá validar que su entorno puede comunicarse correctamente con Amazon Personalize:

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Descargar, preparar y cargar los datos de entrenamiento
Este cuaderno utiliza los datos de las interacciones recopilados del [proyecto de MovieLens](https://grouplens.org/datasets/movielens/). Ejecute las siguientes líneas para descargar la última versión de los datos de MovieLens, importarlos al cuaderno y examinarlos con rapidez.

### Descargar y explorar el conjunto de datos

In [9]:
!wget -N https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip

--2022-04-09 16:48:16--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org... 128.101.65.152
Connecting to files.grouplens.org|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File 'ml-latest-small.zip' not modified on server. Omitting download.

Archive:  ml-latest-small.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [5]:
!ls ml-latest-small

README.txt  links.csv   movies.csv  ratings.csv tags.csv


In [11]:
!pygmentize ml-latest-small/README.txt

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

In [7]:
interactions_data = pd.read_csv('./ml-latest-small/ratings.csv')
pd.set_option('display.max_rows', 5)
interactions_data

userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
...        ...      ...     ...         ...
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]

In [8]:
interactions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


## Preparar los datos

### Datos de las interacciones
Como puede ver, los datos de las interacciones contienen un UserID, un ItemID, una clasificación y una marca temporal. Hay un par de pasos que debe completar antes de poder importar los datos. 

Ejecute el siguiente código para eliminar los elementos con clasificaciones bajas y elimine la columna Rating (Clasificación) antes de importar los datos. También agregue la columna EVENT_TYPE con un tipo de evento Watch para cada interacción.

In [ ]:
interactions_data = interactions_data[interactions_data['rating'] > 3]                # Keep only movies rated higher than 3 out of 5.
interactions_data = interactions_data[['userId', 'movieId', 'timestamp']]
interactions_data.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True)
interactions_data['EVENT_TYPE']='watch' #Adds an EVENT_TYPE column and an event type of "watch" for each interaction.
interactions_data.head()

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

### Metadatos del elemento

Para crear la mejor Magic Movie Machine, también querrá cargar datos sobre cada película. Abra el archivo de datos del elemento y observe las primeras filas.

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)
items_data.info()

In [ ]:
items_data['year'] = items_data['title'].str.extract('.*\((.*)\).*',expand = False)
items_data.head(5)

Dado que se desconoce la marca temporal de creación real de cada película, a continuación se agrega una fecha actual como marca temporal de creación.

In [ ]:
ts= datetime.datetime(2022, 1, 1, 0, 0).strftime('%s')
print(ts)

In [ ]:
items_data["CREATION_TIMESTAMP"] = ts
items_data

In [ ]:
# removing the title
items_data.drop(columns="title", inplace = True)

# renaming the columns to match schema
items_data.rename(columns = { 'movieId':'ITEM_ID', 'genres':'GENRES',
                              'year':'YEAR'}, inplace = True)
items_data

### Metadatos del usuario

Para que Magic Movie Machine ofrezca una personalización basada en los metadatos de los usuarios, querrá importar datos sobre los usuarios. El conjunto de datos de MovieLens no tiene metadatos de usuario, por lo que puede crear metadatos falsos para cada usuario que interactúe con una película de la siguiente manera:

In [ ]:
# get user ids from the interaction dataset

user_ids = interactions_data['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]=user_ids
user_data

#### Incorporación de metadatos
El conjunto de datos actual no contiene información adicional del usuario. Para este ejemplo, asignaremos un género a los usuarios de forma aleatoria con la misma probabilidad de ser hombre o mujer. Este tipo de atributos puede producir recomendaciones más personalizadas.

In [ ]:
possible_genders = ['female', 'male']
random = np.random.choice(possible_genders, len(user_data.index), p=[0.5, 0.5])
user_data["GENDER"] = random
user_data

## Configurar un bucket de S3 y un rol de IAM

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora. Tan solo defina la región como una cadena a continuación.

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled this should be the same region as the Amazon Personalize resources
# region = "us-east-1"


In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedvod"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except:
    print("Bucket already exists. Using bucket", bucket_name)

### Cargar los datos a S3
Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de interacción entre el usuario y los elementos.

In [ ]:
interactions_filename = "interactions.csv"
interactions_data.to_csv(interactions_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

items_filename = "items.csv"
items_data.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

user_filename = "users.csv"
user_data.to_csv(user_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_filename).upload_file(user_filename)

## Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

Nota: Asegúrese de que el rol que está utilizando para ejecutar el código en este cuaderno tiene los permisos necesarios para modificar la política del bucket de S3.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## Crear y esperar el grupo de conjuntos de datos
Ahora es el momento de crear sus recursos de Amazon Personalize. En primer lugar, creará un grupo de conjuntos de datos de dominio con el dominio establecido como VIDEO_ON_DEMAND. Un grupo de conjuntos de datos de dominio es un contenedor que almacena recursos configurados previamente como conjuntos de datos y recomendadores para diferentes dominios empresariales y casos de uso. Siéntase libre de modificar el nombre que aparece a continuación si lo desea.

### Crear grupo de conjuntos de datos

In [ ]:
response = personalize.create_dataset_group(
    name='personalize-video-on-demand-ds-group',
    domain='VIDEO_ON_DEMAND'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

Esperar a que el grupo de conjuntos de datos tenga un estado ACTIVO
Antes de que podamos utilizar el grupo de conjuntos de datos en cualquier elemento, este debe estar activo. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Crear un esquema de interacciones
Un componente central de cómo Personalize entiende sus datos proviene del esquema que se define a continuación. Esta configuración indica al servicio cómo leer los datos proporcionados a través de su archivo CSV. Observe que las columnas y los tipos se alinean con lo que había en el archivo que creó anteriormente.

In [ ]:
schema = {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "EVENT_TYPE",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_interactions_schema_response = personalize.create_schema(
    name='personalize-demo-interactions-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

interactions_schema_arn = create_interactions_schema_response['schemaArn']
print(json.dumps(create_interactions_schema_response, indent=2))

# Crear un esquema para los elementos (películas)

Al igual que antes, crearemos un esquema para los elementos, que en este caso son películas.

In [ ]:
schema = {
  "type": "record",
  "name": "Items",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
    {
      "name": "ITEM_ID",
      "type": "string"
    },
    {
      "name": "GENRES",
      "type": [
        "string"
      ],
      "categorical": True
    },
    {
      "name": "YEAR",
      "type": [
        "string"
      ],
      "categorical": True
    }, 
    {
      "name": "CREATION_TIMESTAMP",
      "type": "long"
    }
  ],
  "version": "1.0"
}
create_items_schema_response = personalize.create_schema(
    name='personalize-demo-items-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

items_schema_arn = create_items_schema_response['schemaArn']
print(json.dumps(create_items_schema_response, indent=2))

# Crear un esquema para los usuarios

Ahora crearemos un esquema para los datos del usuario.

In [ ]:
schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
    ],
    "version": "1.0"
}
create_users_schema_response = personalize.create_schema(
    name='personalize-demo-users-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

users_schema_arn = create_users_schema_response['schemaArn']
print(json.dumps(create_users_schema_response, indent=2))

## Crear conjuntos de datos
La siguiente tarea es utilizar los esquemas que acaba de crear para crear los conjuntos de datos reales de Amazon Personalize dentro de su grupo de conjuntos de datos de dominio. Los conjuntos de datos almacenarán sus datos para el entrenamiento.

### Crear un conjunto de datos de interacciones

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Crear un conjunto de datos para los elementos

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Crear un conjunto de datos para los usuarios

In [ ]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-users",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Crear un rol de Personalize
Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Las siguientes líneas hacen que esto sea posible.

Nota: Asegúrese de que el rol que está utilizando para ejecutar el código en este cuaderno tiene los permisos necesarios para crear un rol. En caso de que no pueda avanzar, consulte la [Guía para desarrolladores de Amazon Personalize](https://docs.aws.amazon.com/personalize/latest/dg/aws-personalize-set-up-permissions.html).

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleVODDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## Importar los datos
Anteriormente creó el grupo de conjuntos de datos de dominio y los conjuntos de datos para alojar los datos de Magic Movie Machine. Ahora ejecutará un trabajo de importación que cargará los datos de S3 en Amazon Personalize para crear su recomendador de películas.
### Crear un trabajo de importación de conjuntos de datos de interacciones

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-interactions",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### Crear un trabajo de importación de conjuntos de datos de elementos

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-items",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### Crear un trabajo de importación de conjuntos de datos de los usuarios

In [ ]:
create_users_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-users",
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_filename)
    },
    roleArn = role_arn
)

dataset_users_import_job_arn = create_users_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_users_dataset_import_job_response, indent=2))

Esperar a que el trabajo de importación de conjuntos de datos tenga un estado ACTIVO.
Puede pasar un tiempo antes de que el trabajo de importación se complete. Espere hasta que vea que está activo a continuación.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Interactions DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Items DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_users_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Users DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Elegir un caso de uso del recomendador

Cada dominio tiene diferentes casos de uso. Cuando se crea un recomendador, se crea para un caso de uso específico, y cada caso de uso tiene diferentes requisitos para obtener recomendaciones.

Para obtener información sobre todos los casos de uso compatibles con Amazon Personalize, consulte [Elección de casos de uso del recomendador](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html).

In [ ]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND') # See a list of recommenders for the domain. 
if (len(available_recipes["recipes"])==0):
    # This is a workaround to get the recipes in case 'available_recipes["recipes"]'does not retrieve them
    available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND', nextToken=available_recipes["nextToken"])
display(available_recipes["recipes"])
    

Crearemos un recomendador del tipo *Más como X*. *Más como X* es una versión más sencilla de las recomendaciones de *Joyas ocultas* que vio en Magic Movie Machine: Genera recomendaciones de películas similares, pero que no son joyas menos conocidas. Con este caso de uso, Amazon Personalize filtra de forma automática los videos que el usuario vio en función del userId especificado en la llamada `get_recommendations`.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'more_like_x_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  datasetGroupArn = dataset_group_arn
)
recommender_more_like_x_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Crearemos un segundo recomendador del tipo "Principales selecciones para usted". Este tipo de recomendador ofrece recomendaciones personalizadas de contenido de streaming para un usuario que especifique. Con este caso de uso, Amazon Personalize filtra automáticamente los videos que el usuario vio en función del userId que especifique y los eventos `Watch`.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'top_picks_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
  datasetGroupArn = dataset_group_arn
)
recommender_top_picks_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

Esperamos a que los recomendadores terminen de crear y tengan estado `ACTIVE`. Comprobamos periódicamente el estado del recomendador.

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_more_like_x_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_more_like_x_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_more_like_x_arn))

    if status == "ACTIVE":
        break
    else:
        print("The More Like X Recommender build is still in progress")
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_top_picks_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_top_picks_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_top_picks_arn))

    if status == "ACTIVE":
        break
    else:
        print("The Top Pics for You Recommender build is still in progress")
        
    time.sleep(60)

## Obtener recomendaciones
Ahora que creó sus recomendadores de películas, veamos las recomendaciones de películas que puede obtener para sus usuarios.

In [ ]:
# reading the original data in order to have a dataframe that has both movie_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./ml-latest-small/movies.csv')
items_df.sample(10)

In [ ]:
def get_movie_by_id(movie_id, movie_df):
    """
    This takes in an movie_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[movie_df["movieId"]==int(movie_id)]['title'].values[0]
    except:
        print (movie_id)
        return "Error obtaining title"

### Obtenga recomendaciones de películas “Más como X”

El recomendador “Más como X” genera recomendaciones de películas similares. Proporciona un ID de una película y Amazon Personalize devuelve una lista de películas similares.

In [ ]:
# First pick a user
test_user_id = "1"

# Select a random item
test_item_id = "81847" #Iron Man 59315, Tangled: 81847

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_more_like_x_arn,
    userId = test_user_id,
    itemId = test_item_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_movie_by_id(test_item_id, items_df)])

pd.options.display.max_rows = 20
display(user_recommendations_df)

### Obtener recomendaciones “Principales selecciones para usted”

Ahora obtendrá recomendaciones personalizadas para un usuario con el recomendador *Principales selecciones para usted*. Primero, utilice el siguiente código para agregar los metadatos del usuario a nuestro usuario de ejemplo. Puede utilizar este tipo de metadatos para obtener información sobre sus usuarios.

In [ ]:
users_data_df = pd.read_csv('./users.csv')

def get_gender_by_id(user_id, user_df):
    """
    This takes in a user_id and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    try:
        return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    except:
        print (user_id)
        return "Error obtaining title"

A continuación, elija un usuario y obtenga recomendaciones de películas para él.

In [ ]:
# First pick a user
test_user_id = "111" # samples users: 55, 75, 76, 111

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_top_picks_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

column_name = test_user_id+" ("+get_gender_by_id(test_user_id, users_data_df)+")"

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [column_name])

pd.options.display.max_rows =20
display(user_recommendations_df)

## Revisión
Con el código anterior, creó con éxito recomendadores que funcionan igual que en Magic Movie Machine: Entrenó un modelo de aprendizaje profundo para generar recomendaciones de películas basadas en el comportamiento previo del usuario. 

Creó dos recomendadores para dos casos de uso básicos: uno para el caso de uso *Principales selecciones para usted* y otro para el caso de uso *Más como X*. En adelante, puede adaptar este código para crear otros recomendadores.

## Notas para el próximo cuaderno:
Hay algunos valores que necesitará para el siguiente cuaderno, ejecute la celda a continuación para almacenarlos y así poder utilizarlos en el cuaderno `Clean_Up_Resources.ipynb`.

Esto sobreescribirá cualquier dato almacenado para esas variables y las ajustará a los valores especificados en este cuaderno. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region